# Algebraic $E_\infty$-bialgebra on cubes

In this notebook we describe a natural $E_\infty$-bialgebra structure on $C_\bullet(\mathrm I^d;\mathbb F_2)$, the cellular chains of a standard cube, using the finitely presented $E_\infty$-prop of [[1]](https://arxiv.org/abs/1808.00854).

## $E_\infty$-coalgebras and the prop $\mathcal S$

Recall...

## Chain on standard cubes
 Recall...

In [ ]:
from itertools import product as times

def basis(d):
    if d == 0:
        return ['0']
    
    letters = ['0','1','e']
    basis = [''.join(ltr) for ltr in times(letters, repeat=d)]
    
    return basis

def boundary(x):
    answer = set()
    for i, letter in enumerate(x):
        if letter == 'e':
            answer ^= {x[:i]+'0'+x[i+1:],
                       x[:i]+'1'+x[i+1:]}
    return answer

def degree(x):
    return x.count('e')

## Coproduct

We will model the function

$$\Delta : C_\bullet(\mathrm I^d;\mathbb F_2) \to C_\bullet(\mathrm I^d;\mathbb F_2) \otimes C_{\bullet}(\mathrm I^d;\mathbb F_2)$$


defined by

$$ \Delta \big( (x_1,\dots x_d) \big) = 
(\pi_1\Delta(x_1), \dots, \pi_1\Delta(x_d)) \otimes
(\pi_1\Delta(x_1), \dots, \pi_1\Delta(x_d))$$

where 
$$\Delta([0]) = [0] \otimes [0] \qquad \Delta([1]) = [1] \otimes [1]$$
and
$$ \Delta(e) = e \otimes [0] + [1] \otimes e.$$

In [ ]:
def coproduct(x, answer=set()):
    if len(x) == 0:
        return {('','')}
    
    if x[0] == 'e':
        
        answer = {('0'+pair[0], 'e'+pair[1]) for pair in coproduct(x[1:], answer)} \
               ^ {('e'+pair[0], '1'+pair[1]) for pair in coproduct(x[1:], answer)}
    else:
        answer = {(x[0]+pair[0], x[0]+pair[1]) for pair in coproduct(x[1:], answer)}
    
    return answer

def display_coproduct(x):
    coprod = str(coproduct(x))
    coprod = coprod.replace("', '","]|[")
    
    space = ' '*(10+len(x))
    coprod = coprod.replace("'), ('","]\n"+space+"+ [")
    
    return f'Delta([{x}]) = ['+coprod[3:-3]+']'

### Example

Given an choice of integer $d$, we print the coproduct $\Delta(x)$ for all basis elements in $C_\bullet(\mathrm I^d; \mathbb F_2)$.

In [ ]:
d = 0

for x in basis(d):
    print(display_coproduct(x))

### Check boundary of coproduct

We define a method that verifies that for any basis element $x \in C_\bullet(\mathrm I^d; \mathbb F_2)$

$$(\partial \otimes \mathrm{id} + \mathrm{id}\otimes \partial)\, \Delta(x) + \Delta(\partial x) = 0$$

In [ ]:
def check_boundary_coproduct(x):
    '''returns True if the boundary the coproduct acting 
    on the basis element x is 0'''
    
    answer = set()
    for term in boundary(x):
        answer ^= coproduct(term)
        
    for pair in coproduct(x):
        bdry_0 = boundary(pair[0])
        answer ^= {(term, pair[1]) for term in bdry_0}
        
        bdry_1 = boundary(pair[1])
        answer ^= {(pair[0], term) for term in bdry_1}
        
    return not bool(answer)

## Counit

We will model the function 

$$\varepsilon : C_\bullet(\mathrm I^d;\mathbb F_2) \to \mathbb F_2$$ 

defined on basis elements by

$$\varepsilon(x) = 
\begin{cases}
1 & |x| = 0 \\
0 & |x| > 0.
\end{cases}$$

In [ ]:
def counit(x):
    if degree(x) > 0:
        return 0
    if degree(x) == 0:
        return 1

### Check counit property

We define a method that verifies that for any basi element $x \in C_\bullet(\mathrm I^d; \mathbb F_2)$

$$(\varepsilon(x) \otimes \mathrm{id})\,\Delta(x) = x = (\mathrm{id} \otimes \varepsilon(x))\,\Delta(x).$$

In [ ]:
def check_counit_property(x):
    '''return True if applying the counit to any of the factors 
    of the coproduct of any basis element gives the element back'''
    coprod = coproduct(x)
    a = {pair[1] for pair in coprod if bool(counit(pair[0]))}
    a ^= {x}
    b = {pair[0] for pair in coprod if bool(counit(pair[1]))}    
    b ^= {x}
    return not (bool(a) or bool(b))

## Product

We will model the function
$$\ast : C_\bullet(\mathrm I^d;\mathbb F_2) \otimes C_\bullet(\mathrm I^d;\mathbb F_2) \to C_{\bullet+1}(\mathrm I^d;\mathbb F_2)$$

defined by

$$ \ast \big( (x_1,\dots x_d) \otimes (y_1,\dots y_d) \big) = 
\sum_{i=1}^d \big( y_1,\dots,y_{i-1}, (x_i\ast y_i), y_{i+1}, \dots, y_d \big).$$

where $$\ast(x,y) = \begin{cases} 
e &  x=[0]\ y=[1] \text{ or} \\
  &  x=[1]\ y=[0] \\
0 & \text{otherwise}.
\end{cases}$$

In [ ]:
_ast = {'01':'e','10':'e'}

def product(x,y):
    assert len(x)== len(y), 'not in the same cube'
    
    answer = set()
    for i, pair in enumerate(zip(x,y)):
        try:
            answer ^= {y[:i]
                      +_ast[pair[0]+pair[1]]
                      +x[i+1:]}
        except KeyError:
            pass
    
    expected_degree = degree(x)+degree(y)+1
    answer = {term for term in answer if 
              term.count('e') == expected_degree}
    
    return answer

def display_product(x,y):
    jn = product(x,y)
    
    if jn:
        return f'[{x}]*[{y}] = [' + ']+['.join(jn) + ']'
    if not jn:
        return f'[{x}]*[{y}] = 0'

### Example
Given a choice of integer $d$, we print the product $x*y$ for all pairs of basis elements in $C_\bullet(\mathrm I^d; \mathbb F_2)$.

In [ ]:
d = 1
only_nonzero = True

for x,y in times(basis(d), repeat=2):
    if product(x,y) or not only_nonzero:
        print(display_product(x,y))

### Check boundary of the product

We define a method that verifies that for any pair of basis elements $x,y \in C_\bullet(\mathrm I^d; \mathbb F_2)$

$$\partial (x \ast y) + (\partial x) \ast y + x \ast (\partial y) 
= \varepsilon(x)y + \varepsilon(y)x.$$

In [ ]:
def check_product_boundary(x,y):
    '''returns True if the boundary of the product
    acts on a pair of basis elements x and y as the 
    sum of the aumentation()'''
    answer = set()
    for x_term in boundary(x):
        answer ^= product(x_term, y)

    for y_term in boundary(y):
        answer ^= product(x, y_term)

    for z in product(x,y):
        answer ^= boundary(z)

    if bool(counit(x)):
        answer ^= {y}

    if bool(counit(y)):
        answer ^= {x}

    return not bool(answer)